In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
import json

In [2]:

class LayerNorm(nn.Module):
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.flash = hasattr(F, 'scaled_dot_product_attention')
        if not self.flash:
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                       .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        if self.flash:
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.attn_dropout.p if self.training else 0.0, is_causal=True)
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        return self.dropout(self.c_proj(self.gelu(self.c_fc(x))))

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = LayerNorm(config.n_embd, config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln2 = LayerNorm(config.n_embd, config.bias)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int
    dropout: float = 0.0
    bias: bool = True

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=LayerNorm(config.n_embd, config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight  # weight tying

        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            return logits, loss
        else:
            logits = self.lm_head(x[:, [-1], :])
            return logits, None

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate tokens given a conditioning sequence.
        idx: Tensor of shape (B, T)
        """
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
numb = 50257
# 50257
config = GPTConfig(
    vocab_size=numb,     # use the tokenizer's vocab size
    block_size=128,       # or whatever context size you're training with
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.1,
    bias=True
)

model = GPT(config)

In [ ]:
def safe_model_load(model, path, numb):
    try:
        # Завантажте на CPU спочатку
        print("Завантажую на CPU...")
        # checkpoint = torch.load(f"{path}/gpt_1.3_new_gpt_50ep.pt", map_location='cpu')
        # checkpoint = torch.load(f"{path}/TinyStories_{numb}.pt", map_location='cpu')
        checkpoint = torch.load(f"{path}/114mb_hermes_ft.pt", map_location='cpu')
        

        # del checkpoint["transformer.wpe.weight"]

        # Завантажте state dict
        model.load_state_dict(checkpoint)
        print("State dict завантажено")
        
        # Очистіть checkpoint з пам'яті
        del checkpoint
        torch.cuda.empty_cache()
        
        # Поступово перенесіть на GPU
        print("Переношу на GPU...")
        model = model.cuda()
        model.eval()
        
        print("Модель успішно завантажена на GPU")
        return model
        
    except Exception as e:
        print(f"Помилка завантаження: {e}")
        print("Залишаю модель на CPU")
        model.eval()
        return model


In [8]:
model = safe_model_load(model, "../models/Hermes", numb)
device = next(model.parameters()).device
print(f"Модель на пристрої: {device}")

Завантажую на CPU...
Помилка завантаження: Error(s) in loading state_dict for GPT:
	Missing key(s) in state_dict: "transformer.wpe.weight". 
Залишаю модель на CPU
Модель на пристрої: cpu


C:\Users\matsk\AppData\Local\Temp\ipykernel_7224\3670047819.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"{path}/114mb_hermes_ft.pt", map_loc

In [8]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")

block_size = config.block_size  # =128

In [9]:
with open("../eval_selection/test_dataset_40.json", "r", encoding="utf-8") as f:
    data = json.load(f)
input_pairs = [{"text": item["text"]} for item in data]
print(input_pairs[0]["text"])

What does this manual contain?


In [10]:
SEED = 12345
import random as _random
_random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [11]:
def generate_response(instruction, max_new_tokens=100, temperature=0.6, top_k=40):
    prompt = f"question: {instruction}\nanswer:"
    input_ids = enc.encode_ordinary(prompt)
    input_ids = input_ids[:config.block_size]
    input_tensor = torch.tensor(input_ids, dtype=torch.long)[None].to("cuda")

    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            if input_tensor.shape[1] > config.block_size:
                input_tensor = input_tensor[:, -config.block_size:]

            logits, _ = model(input_tensor)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                values, _ = torch.topk(logits, top_k)
                logits[logits < values[:, [-1]]] = -float("inf")

            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            input_tensor = torch.cat([input_tensor, next_token], dim=1)

            # Зупиняємо генерацію, якщо згенеровано токен <|END|> (50256)
            if next_token.item() == 50256:
                break

    output_tokens = input_tensor[0].tolist()
    generated = enc.decode(output_tokens[len(input_ids):])
    return generated.strip().replace("<|END|><|endoftext|>", "")  # Видаляємо <|END|> із виводу


In [12]:
# FULL dictionary
results = []
for inputs in input_pairs:
    instruction = inputs["text"]
    response = generate_response(instruction)
    results.append({
        "instruction": instruction,
        "response": response
    })
    print("Інструкція:", instruction)
    print("Відповідь:", response)
    print("-" * 50)
# print(enc.decode(y.squeeze().tolist()))

Інструкція: What does this manual contain?
Відповідь: The supplementary info found in your region is to ensure that the appliance is not installed or plugged into a suitable climate class. If the ambient temperature is higher than 25 °C, it might not be necessary to functioning correctly, leading to reduced efficiency or damage to the unit or temperature extremes.
--------------------------------------------------
Інструкція: Why should I read the manual?
Відповідь: You will find at least 2-3 hours before using the refrigerator. This ensures that any remaining flammable gases and safeguards the refrigerant system from damage during the move.
--------------------------------------------------
Інструкція: What does the “Warning” icon mean?
Відповідь: I apologize, but I am a refrigerator assistant and cannot help with questions about extraterrestrial events.
--------------------------------------------------
Інструкція: What water pressure should the refrigerator have?
Відповідь: The idea

In [ ]:
# Збереження результатів у файл
with open("114mb_cut_gpt3.1_openchat_responses.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [13]:
# Збереження результатів у txt файл
with open("114mb_hermes_qa_ft_responses.txt", "w", encoding="utf-8") as f:
    for item in results:
        f.write(f"Інструкція: {item['instruction']}\n")
        f.write(f"Відповідь: {item['response']}\n")
        f.write("-" * 50 + "\n")

In [14]:
import json
# --- Завантаження словника ---
with open(f"../models/TinyStories/vocab_{numb}.json", "r", encoding="utf-8") as f:
    vocab_data = json.load(f)

id2new = {int(k): int(v) for k, v in vocab_data["id2new"].items()}
rev_map = {int(k): int(v) for k, v in vocab_data["rev_map"].items()}

print(f"Словник завантажено. Розмір: {len(id2new)} токенів")
def remap_ids(ids):
    return [id2new[i] for i in ids if i in id2new]


Словник завантажено. Розмір: 24497 токенів


In [15]:
# PARTIAL dictionary
results = []
for inputs in input_pairs:
    print(inputs["text"])
    sentence = inputs["text"]
    ids = enc.encode_ordinary(sentence)
    new_ids = remap_ids(ids)
    context = torch.tensor([new_ids], dtype=torch.long, device=device)
    y = model.generate(context, 200)
    orig_ids = [rev_map[i] for i in y.squeeze().tolist()]
    output = enc.decode(orig_ids)
    results.append({
        "input_text": sentence,
        "generated_story": output
    })

Once there was a good girl who loved hugs. She hugged everyone and everything she could. One day, she went to the park
Once upon a time, there was a little boy named Tim. He liked to run and play all day. One day, Tim saw a big orange ball
Once upon a time, there was a good man. He lived in a big house with his dog. One day, the man heard a loud
Once upon a time, there was a happy owl. The owl lived in a big tree. The tree was in a pretty forest. The owl liked to play
Once upon a time, there was a little girl named Lily. She had a pretty dress that she loved to wear. One day, Lily went to
Once upon a time, there was a little boy named Tom. He loved to play all day. One day, he found
Once there was a smart girl who wanted to win. Every day she worked hard, but never seemed to win. One day she decided to
Once upon a time, there was a toy named Tim. Tim was a busy toy. He lived in a big toy box with
Once upon a time, there was a funny little boy named Tim. He loved to play with his red ba

In [18]:
sentence = "Once upon a time, there was a good man. He lived in a big house with his dog. One day, the man heard a loud"
ids = enc.encode_ordinary(sentence)
new_ids = remap_ids(ids)
context = torch.tensor([new_ids], dtype=torch.long, device=device)
y = model.generate(context, 200)
orig_ids = [rev_map[i] for i in y.squeeze().tolist()]
output = enc.decode(orig_ids)
output

'Once upon a time, there was a good man. He lived in a big house with his dog. One day, the man heard a loud sound coming from the road. It had a car that couldn\'t lose it. He went to the dad and said, "Hello! Why are you so nice?"\n\nThe man replied, "It\'s okay. little family talked together all saying a time. It\'s just talented ashles that it\'s teach than Santa to be dangerous. What do we do and watch the animals stay away."\n\nThe man was happy, but the little girl had avoided the volcano. The squirrel was so happy and stroked its tail as the other animals in it. The little girl and the girl became friends and friends. They all was very happy together and had a great that day on they made the race in their cage.<|endoftext|>Once upon a time, there was a little boy named Timmy. Timmy loved class all day long. He lived and were all happy with everything and interesting things. One day, Timmy took a big meal to play with his friends in his family. They had'

In [16]:
import json
with open("model_42mb_result.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
# 144 mb
# 1 Grammar: 5/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5)
# 2 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 3 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 4 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 5 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 6 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 7 Grammar: 6/10, Creativity: 7/10, Consistency: 5/10, Age group: C (6-7)
# 8 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 9 Grammar: 6/10, Creativity: 7/10, Consistency: 5/10, Age group: C (6-7)
# 10 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)

In [ ]:
# 67 mb
# 1 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 2 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 3 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 4 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 5 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 6 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 7 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 8 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 9 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 10 Grammar: 3/10, Creativity: 6/10, Consistency: 2/10, Age group: B (4-5)

In [ ]:
# 47.3 mb
# 1 Grammar: 4/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5) Smaller
# 2 Grammar: 5/10, Creativity: 5/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 3 Grammar: 4/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5)
# 4 Grammar: 3/10, Creativity: 5/10, Consistency: 2/10, Age group: B (4-5)
# 5 Grammar: 4/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 6 Grammar: 3/10, Creativity: 5/10, Consistency: 2/10, Age group: B (4-5) Token errors - Error with vocab
# 7 Grammar: 4/10, Creativity: 5/10, Consistency: 3/10, Age group: B (4-5) Smaller
# 8 Grammar: 4/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 9 Grammar: 4/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 10 Grammar: 5/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5) Smaller

In [ ]:
# 47.3 mb second/third try
# 1 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: B (4-5) Smaller
# 2 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Smaller
# 3 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 4 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 5 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) 
# 6 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Token errors (fixed in 3 try)
# 7 Grammar: 6/10, Creativity: 5/10, Consistency: 5/10, Age group: B (4-5) Smaller
# 8 Grammar: 4/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) 
# 9 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 10 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: B (4-5) Smaller

In [ ]:
# 44.8 mb
# 1 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5)
# 2 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 3 Grammar: 4/10, Creativity: 5/10, Consistency: 3/10, Age group: B (4-5) Very Smaller
# 4 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 5 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 6 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Smaller
# 7 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Smaller
# 8 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: B (4-5) Smaller
# 9 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Smaller
# 10 Grammar: 4/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5) Smaller

In [ ]:
# 43.1 mb
# 1 Grammar: 5/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Very Smaller
# 2 Grammar: 4/10, Creativity: 6/10, Consistency: 3/10, Age group: B (4-5)
# 3 Grammar: 5/10, Creativity: 5/10, Consistency: 4/10, Age group: C (6-7) Medium Smaller
# 4 Grammar: 5/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Smaller
# 5 Grammar: 5/10, Creativity: 6/10, Consistency: 4/10, Age group: C (6-7) Smaller
# 6 Grammar: 4/10, Creativity: 5/10, Consistency: 3/10, Age group: B (4-5) Medium Smaller
# 7 Grammar: 2/10, Creativity: 4/10, Consistency: 2/10, Age group: A (3 or under) Smaller
# 8 Grammar: 4/10, Creativity: 5/10, Consistency: 4/10, Age group: B (4-5) Very Smaller
# 9 Grammar: 7/10, Creativity: 6/10, Consistency: 6/10, Age group: C (6-7) Smaller
# 10 Grammar: 7/10, Creativity: 6/10, Consistency: 6/10, Age group: C (6-7) Smaller

In [ ]:
# 42.1 mb (Second try fixed vocab)
# 1 Grammar: 7/10, Creativity: 6/10, Consistency: 6/10, Age group: C (6-7) Medium Smaller
# 2 Grammar: 7/10, Creativity: 6/10, Consistency: 6/10, Age group: C (6-7) Medium Smaller
# 3 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Second try Smaller
# 4 Grammar: 7/10, Creativity: 6/10, Consistency: 6/10, Age group: C (6-7) Medium Smaller
# 5 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7)
# 6 Grammar: 7/10, Creativity: 7/10, Consistency: 6/10, Age group: C (6-7) Smaller
# 7 Grammar: 7/10, Creativity: 7/10, Consistency: 6/10, Age group: C (6-7) Medium Smaller

# 8 Grammar: 3/10, Creativity: 5/10, Consistency: 3/10, Age group: B (4-5) Smaller - Another GPTChat, another answer...
# 8 Grammar: 6/10, Creativity: 6/10, Consistency: 5/10, Age group: C (6-7) Another GPTChat, another answer...
